# Read in Data

In [148]:
import pandas as pd

data = pd.read_csv('website/player_visualization.csv', sep=',')
data = data.dropna()

data['amount'] = data['amount'].str.replace('$', '')
data['amount'] = data['amount'].str.replace(',', '')
data['amount'] = data['amount'].astype(float)

data

,Unnamed: 0,player,player_id,yearID,pos,WAR,length,amount,out_of_contract
0,0,CJ Abrams,abramcj01,2023.0,h,3.4,1.0,724200.0,False
1,1,CJ Abrams,abramcj01,2022.0,h,0.1,1.0,724200.0,True
2,2,Jose Abreu,abreujo02,2023.0,h,0.0,3.0,58500000.0,False
3,3,Jose Abreu,abreujo02,2022.0,h,4.3,3.0,58500000.0,True
4,4,Jose Abreu,abreujo02,2021.0,h,2.9,3.0,58500000.0,True
...,...,...,...,...,...,...,...,...,...
46231,46231,Ivan de Jesus,dejesiv01,1984.0,h,0.6,1.0,480.0,True
46232,46232,Ivan de Jesus,dejesiv01,1983.0,h,1.1,1.0,480.0,True
46233,46233,Ivan de Jesus,dejesiv01,1982.0,h,1.3,1.0,480.0,True
46234,46234,Ivan de Jesus,dejesiv01,1981.0,h,-1.3,1.0,480.0,True


In [149]:
# # sum the players total WAR
removal = data.groupby('player')['WAR'].sum().sort_values(ascending=False)

removal = pd.DataFrame(removal)

removal

#get players with above 5 career WAR
removal = removal[removal['WAR'] >= 7.6]
removal


merged = pd.merge(data, removal, on='player', how='inner')

merged

data = merged

# change the headr from WAR_y to total_WAR
data = data.rename(columns={'WAR_y': 'total_WAR'})

data = data.rename(columns={'WAR_x': 'WAR'})

data

,Unnamed: 0,player,player_id,yearID,pos,WAR,length,amount,out_of_contract,total_WAR
0,2,Jose Abreu,abreujo02,2023.0,h,0.0,3.0,58500000.0,False,32.0
1,3,Jose Abreu,abreujo02,2022.0,h,4.3,3.0,58500000.0,True,32.0
2,4,Jose Abreu,abreujo02,2021.0,h,2.9,3.0,58500000.0,True,32.0
3,5,Jose Abreu,abreujo02,2020.0,h,3.0,3.0,58500000.0,True,32.0
4,6,Jose Abreu,abreujo02,2019.0,h,2.5,3.0,58500000.0,True,32.0
...,...,...,...,...,...,...,...,...,...,...
5797,41290,Bo Jackson,jacksbo01,1990.0,h,3.5,1.0,1000000.0,True,8.4
5798,41291,Bo Jackson,jacksbo01,1989.0,h,2.7,1.0,1000000.0,True,8.4
5799,41292,Bo Jackson,jacksbo01,1988.0,h,1.7,1.0,1000000.0,True,8.4
5800,41293,Bo Jackson,jacksbo01,1987.0,h,0.0,1.0,1000000.0,True,8.4


In [150]:
length_filter = data
length_filter = length_filter[length_filter['length'] > 6]

# one = length_filter[length_filter['length'] == 4]
# two = length_filter[length_filter['length'] == 5]
# three = length_filter[length_filter['length'] == 6]

# merged = pd.concat([one, two, three])

# length_filter = merged

data = length_filter

# Run T Test

In [151]:
from scipy import stats

# Group the data by 'player_id'
grouped = data.groupby('player_id')


# Test players individually

In [152]:
# Initialize a dictionary to store the t-test results for each player
t_test_results = {}

# Perform the t-test for each player
for player_id, group in grouped:
    group1 = group[group['out_of_contract'] == True]['WAR']
    group2 = group[group['out_of_contract'] == False]['WAR']

    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy='omit')
    t_test_results[player_id] = (t_statistic, p_value)

# Print the t-test results
not_significant = {}
significant = {}
very_significant = {}
highly_significant = {}


for player_id, (t_statistic, p_value) in t_test_results.items():
    if p_value > 0.05:
        not_significant[player_id] = (t_statistic, p_value)
    
    elif p_value <= 0.5 and p_value > 0.01:
        significant[player_id] = (t_statistic, p_value)
    
    elif p_value <= 0.01 and p_value > 0.001:
        very_significant[player_id] = (t_statistic, p_value)
    
    elif p_value <= 0.001 and p_value > 0.0001:
        highly_significant[player_id] = (t_statistic, p_value)
    
    else:
        pass

print('Not significant: ' + str(round(len(not_significant)/len(t_test_results)*100, 2)) + '%')
print('Significant: ' + str(round(len(significant)/len(t_test_results)*100, 2)) + '%')
print('Very significant: ' + str(round(len(very_significant)/len(t_test_results)*100, 2)) + '%')
print('Highly significant: ' + str(round(len(highly_significant)/len(t_test_results)*100, 2)) + '%')

print(len(not_significant) + len(significant) + len(very_significant) + len(highly_significant))

Not significant: 82.54%
Significant: 7.94%
Very significant: 3.17%
Highly significant: 1.59%
60


In [153]:
print('Not significant: ' + str(len(not_significant)))
print('Significant: ' + str(len(significant)))
print('Very significant: ' + str(len(very_significant)))
print('Highly significant: ' + str(len(highly_significant)))

print(very_significant)
print(highly_significant)



Not significant: 52
Significant: 5
Very significant: 2
Highly significant: 1
{'griffke02': (3.2911023256471044, 0.003650148965920169), 'swansda01': (-4.5612667800924855, 0.0038457676329735797)}
{'rizzoan01': (-4.736710950873315, 0.0006126839588243986)}


# test all players together

In [154]:
from matplotlib import pyplot as plt
# plt.hist(group1, label='Out of Contract', alpha=0.5)
# plt.hist(group2, label='In Contract', alpha=0.5)
# plt.legend()

# Boxplot for group1 and group2 

#plt.boxplot([group1, group2], labels=['Out of Contract', 'In Contract'])

In [155]:
group1 = data[data['out_of_contract'] == True]['WAR']
group2 = data[data['out_of_contract'] == False]['WAR']

t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy='omit')

print(f"T-statistic: {t_statistic}, P-value: {p_value}")

T-statistic: 1.459736291265558, P-value: 0.1447693533372134


In [156]:
t_test_groups = [not_significant, significant, very_significant, highly_significant]
grrr = 0
rawrrr = 0
for group in t_test_groups:
    total_amount = 0
    total_length = 0
    for name in group:
        tables = data[data['player_id'] == name]
        amount = tables['amount'].iloc[0]
        length = tables['length'].iloc[0]
       
        per_year = amount/length
        per_year = '${:,.2f}'.format(per_year)
        #print(tables['player'].iloc[0] + ' made ' + str(per_year))

        total_amount += amount
        total_length += length

        avg = total_amount/total_length
        avg = '${:,.2f}'.format(avg)
    grrr += total_amount
    rawrrr += total_length
    print(avg + ' is the average amount per year for the group')

grrr = '${:,.2f}'.format(grrr/rawrrr)

print('')
print(grrr)


    
    

$22,545,045.05 is the average amount per year for the group
$22,890,243.90 is the average amount per year for the group
$18,312,500.00 is the average amount per year for the group
$5,857,142.86 is the average amount per year for the group

$22,209,645.67


In [157]:

for name in significant:
    tables = data[data['player_id'] == name]
    amount = tables['amount'].iloc[0]
    length = tables['length'].iloc[0]
    per_year = amount/length
    per_year = '${:,.2f}'.format(per_year)
    print(tables['player'].iloc[0] + ' made ' + str(per_year))

Kris Bryant made $26,000,000.00
KeBryan Hayes made $8,750,000.00
Marcus Semien made $25,000,000.00
Mike Trout made $35,541,666.67
Chase Utley made $12,142,857.14
